In [2]:
import mysql.connector
import config
import time
import requests
import json

#creates and opens a connection to the AWS database
def create_cnx_cur():
    cnx = mysql.connector.connect(
            host = config.host,
            user = config.user,
            passwd = config.password)

    cursor = cnx.cursor()
    return (cnx, cursor)


def close_connection(cnx, cursor):
    cursor.close()
    cnx.close()

#creates a new database in the AWS database
def create_database(cursor, database):
    try:
        #it will try to create a database with whatever name passed through
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
        #if this fails, the error will print out as a message
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        try:
            exit()
        except NameError:
            return

    #this is going to try the above function
    try:
        cursor.execute("USE {}".format(database))
    except mysql.connector.Error as err:
        print("Database {} does not exists.".format(database))
        if err.errno == errorcode.ER_BAD_DB_ERROR:
            create_database(cursor, database)
            print("Database {} created successfully.".format(database))
            cnx.database = database
        else:
            print(err)
            exit()


def add_tables_to_db(TABLES, cursor, db_name):
    from mysql.connector import errorcode
    for table_name in TABLES:
        table_description = TABLES[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(f"""USE {db_name}""")
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")

In [3]:
c = create_cnx_cur()

In [4]:
cnx = c[0]
cursor = c[1]

In [5]:
db_name = 'Moovies'

In [ ]:
create_database(cursor, db_name)

In [21]:
TABLES = {}

# TABLES['movies'] = ("""CREATE TABLE movies (
#                                  movieId VARCHAR(25),
#                                  movieTitle VARCHAR(255) NOT NULL,
#                                  revenue REAL,
#                                  rating REAL,
#                                  released DATE NOT NULL,
#                                  production TEXT,
#                                  PRIMARY KEY (movieTitle, released)
#                                  ) ENGINE=InnoDB""")

TABLES['paul'] = ("""CREATE TABLE paul (
                            title VARCHAR(255) NOT NULL,
                            released DATE NOT NULL,
                            imdbID VARCHAR(25),
                            director VARCHAR(50),
                            PRIMARY KEY (title, released)
                            ) ENGINE=InnoDB""")

TABLES['actors'] = ("""CREATE TABLE actors (
                            id INTEGER AUTO_INCREMENT,
                            imdbid VARCHAR(25),
                            actor VARCHAR(225),
                            PRIMARY KEY (id)
                            ) ENGINE=InnoDB""")

In [22]:
add_tables_to_db(TABLES, cursor, db_name)

Creating table paul: already exists.
Creating table actors: OK


In [16]:
close_connection(cnx,cursor)

In [ ]:
def add_businesses_to_db(tryresults):
    cnx_cur = create_cnx_cur()
    cnx = cnx_cur[0]
    cursor = cnx_cur[1]
    # for each business that is in our results from our API request
    for i in range(len(tryresults['businesses'])):
        # sql statement to add row
        add_business = ("""INSERT INTO businesses
                                VALUES (%s, %s, %s, %s, %s)
                        """)
        try:
            # note: location is made of a two-item list -- the first address line
            # and the city/state/etc.
            business_values = (tryresults['businesses'][i]['id'],
                            tryresults['businesses'][i]['name'],
                            tryresults['businesses'][i]['price'],
                            tryresults['businesses'][i]['rating'],
                            tryresults['businesses'][i]['location']['display_address'][0]+", "+tryresults['businesses'][i]['location']['display_address'][1])

            cursor.execute("USE yelp") #gets us into the yelp db instance
            cursor.execute(add_business, business_values)

        except mysql.connector.IntegrityError: # here to skip existing entries
            continue
        except KeyError: # here to skip the restaurants that are missing values, such as price
            continue
        except IndexError:
            try:
                print("Issue with: ", tryresults['businesses'][i])
                business_values = (tryresults['businesses'][i]['id'],
                                tryresults['businesses'][i]['name'],
                               tryresults['businesses'][i]['price'],
                               tryresults['businesses'][i]['rating'],
                               tryresults['businesses'][i]['location']['display_address'][0])
                cursor.execute("USE yelp") #gets us into the yelp db instance
                cursor.execute(add_business, business_values)
            except:
                print("Issue with: ", tryresults['businesses'][i])
                business_values = (tryresults['businesses'][i]['id'],
                                tryresults['businesses'][i]['name'],
                               tryresults['businesses'][i]['price'],
                               tryresults['businesses'][i]['rating'],
                               null)
                cursor.execute("USE yelp") #gets us into the yelp db instance
                cursor.execute(add_business, business_values)




    cnx.commit()
    close_connection(cnx, cursor)